<a href="https://colab.research.google.com/github/yzbaya/FAD.Machine-Learning/blob/main/FAD_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [12]:
data=pd.read_csv('FAD.csv')

<ipython-input-12-d0f901534e20>:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('FAD.csv')


**Préparation des Données **

In [13]:
#Conversion Data
data['DDA'] = pd.to_datetime(data['DDA'], errors='coerce', dayfirst=True)
data['DDV'] = pd.to_datetime(data['DDV'], errors='coerce', dayfirst=True)

In [15]:
#selectionner les colonnes spécifique
selected_columns=[
    'CMJ',
    # 'DDA',
    # 'DDV',
    'STDA',
    'NJR',
    ]

#séparation des colonnes numériques et non numériques
numeric_cols=data.select_dtypes(include=[np.number]).columns
non_numeric_cols = data.select_dtypes(exclude=[np.number]).columns
#creation des dataframes séparer
numeric_data=data[numeric_cols]
non_numeric_data=data[non_numeric_cols]
#imputation pour des valeur numérique
imputer_num=SimpleImputer(strategy='median')
numeric_data_imputed=pd.DataFrame(imputer_num.fit_transform(numeric_data)).
